In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

import pandas as pd
import numpy as np
import joblib

import mtglobals

In [4]:
client, mt_env = mtglobals.gen_client()

Your account balance is 500000.00


### (a) Load the stage-1 task XML data

In [3]:
# The question we ask the workers is contained in this file.
with open("question_stage1.xml", "r", encoding='utf-8') as f:
    stage1_question = f.read()

### (b) Set up the two requirements for the stage-1 task

(The first is that they're in the US, the second that they have *not* been assigned the `Workplace_Survey_Participant` qual before)

In [6]:
# Example of using qualification to restrict responses to Workers who have had
# at least 80% of their assignments approved. See:
# http://docs.aws.amazon.com/AWSMechTurk/latest/AWSMturkAPI/ApiReference_QualificationRequirementDataStructureArticle.html#ApiReference_QualificationType-IDs
stage1_requirements = [
    {
        # MTurk-provided qual id for restricting by country
        'QualificationTypeId':"00000000000000000071",
        'Comparator':"EqualTo",
        'LocaleValues':[{
            'Country':"US"
        }]
    },
    {
        # Workplace_Survey_Participant qual id
        'QualificationTypeId': mtglobals.participant_qual_id,
        'Comparator': 'DoesNotExist',
    }
]

### (c) Create and launch the HIT via the API

In [7]:
# Create the HIT
response = client.create_hit(
    MaxAssignments=mtglobals.num_hits,
    LifetimeInSeconds=86400, # 24 hours
    AssignmentDurationInSeconds=3600, # 1 hour
    Reward=mt_env['reward'],
    Title='Quick 3-question survey about work [<15 seconds], instantly unlock 2nd-stage HIT with higher reward',
    Keywords='survey,workplace,work',
    Description='Quick <15 second survey about workplace conditions, upon submission will receive email with 2nd-stage HIT with higher reward',
    Question=stage1_question,
    QualificationRequirements=stage1_requirements,
)

### (d) Get the links to the stage-1 HIT (for verification, if need be)

In [8]:
# The response included several fields that will be helpful later
hit_type_id = response['HIT']['HITTypeId']
hit_id = response['HIT']['HITId']
print("\nCreated {} HITs, with HITId: {}".format(num_hits, hit_id))

print("\nYou can work the HIT here:")
print(mt_env['preview'] + "?groupId={}".format(hit_type_id))

print("\nAnd see results here:")
print(mt_env['manage'])


Created 1000 HITs, with HITId: 3BCRDCM0OEV9AO8YCYWZ0G46CS4K6A

You can work the HIT here:
https://www.mturk.com/mturk/preview?groupId=3N79SUTLM3MWO9D9UUEITN063OM5BI

And see results here:
https://requester.mturk.com/mturk/manageHITs


### (e) Save the HIT id for the newly-created stage-1 HIT

(just to make the next step a little easier)

In [4]:
# Save the id of the newly-created stage 1 HIT
joblib.dump(hit_id, "stage1_hit_id.pkl")

NameError: name 'hit_id' is not defined